In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np


In [2]:
a = -float('inf')
print(1+a)

-inf


In [3]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(f'Ongoing device : {device}')

Ongoing device : cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize(784),
    transforms.ToTensor(), 
    transforms.Normalize((0.5,),(0.5,))
    ])

In [5]:
mnist_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transform,
                               download=True)

data_loader = DataLoader(dataset=mnist_dataset,
                         batch_size=100,
                         shuffle=True)

In [6]:
class Discriminator(nn.Module):
    def __init__(self,image_dim=784,hidden_dim=256,latent_dim=1):
        super(Discriminator,self).__init__()

        self.fc = nn.Sequential(nn.Linear(image_dim,hidden_dim), 
                                nn.ReLU(), 
                                nn.Linear(hidden_dim,latent_dim), 
                                nn.Sigmoid()
                                )
        
    def forward(self,x):
        x = self.fc(x)
        return x
        

        
class Generator(nn.Module):
    def __init__(self,image_dim=784,hidden_dim=256,latent_dim=100):
        super(Generator,self).__init__()

        self.fc = nn.Sequential(nn.Linear(latent_dim,hidden_dim), 
                                nn.LeakyReLU(0.2), 
                                nn.Linear(hidden_dim,image_dim), 
                                nn.Tanh()
                                )
        
    def forward(self,x):
        x = self.fc(x)
        return x

In [7]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)

In [8]:
loss_fn = nn.BCEWithLogitsLoss()
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

In [15]:
# Set the number of epochs and the noise size
num_epochs = 200
noise_size = 100

# Training loop
for epoch in range(num_epochs):
  for i, (real_images, _) in enumerate(data_loader):
    # Get the batch size
    batch_size = real_images.size(0)
    # Generate fake images
    noise = torch.randn(batch_size, noise_size).to(device)
    fake_images = generator(noise)
  
    # Train the discriminator on real and fake images
    d_real = discriminator(real_images.to(device))
    d_fake = discriminator(fake_images.to(device))
  
    # Calculate the loss
    real_loss = loss_fn(d_real, torch.ones_like(d_real).to(device))
    fake_loss = loss_fn(d_fake, torch.zeros_like(d_fake).to(device))
    d_loss = real_loss + fake_loss
  
    # Backpropagate and optimize
    d_optimizer.zero_grad()
    d_loss.backward(retain_graph=True)
    d_optimizer.step()
  
    # Train the generator
    d_fake = discriminator(fake_images)
    g_loss = loss_fn(d_fake, torch.ones_like(d_fake).to(device))
  
    # Backpropagate and optimize
    g_optimizer.zero_grad()
    g_loss.backward()
    g_optimizer.step()
  
    # Print the loss every 50 batches
    if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(data_loader)}], '
                  f'Loss_D: {d_loss.item() + d_loss.item():.4f}, Loss_G: {g_loss.item():.4f}')

Epoch [1/200], Batch [100/600], Loss_D: 2.6719, Loss_G: 0.5039
Epoch [1/200], Batch [200/600], Loss_D: 2.0387, Loss_G: 0.6838
Epoch [1/200], Batch [300/600], Loss_D: 2.3980, Loss_G: 0.5872
Epoch [1/200], Batch [400/600], Loss_D: 2.1295, Loss_G: 0.6528
Epoch [1/200], Batch [500/600], Loss_D: 2.1108, Loss_G: 0.6612
Epoch [1/200], Batch [600/600], Loss_D: 2.1807, Loss_G: 0.6477


KeyboardInterrupt: 

In [18]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 5060 Ti


In [19]:
print(torch.cuda.device_count())

1
